In [1]:
import sys 
from aima3 import learning
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import torch
import torch.nn as nn 
import torch.nn.functional as func 
from torch.autograd import Variable
from torchviz import make_dot 

### Backpropagation and the Chain Rule


*Foreword: In this notebook we use slightly different terminology. An arbitrary training instance is denoted as $(v, y) \in E$ where $v$ is the collection of predictors, $y$ is the target, and $E$ is the training set. Moreover, the network weights are denoted by $x$.* 

Deep learning is fundamentally a giant problem in optimisation. We are choosing numerical "weights" to minimise a loss function $L$ (which depends on those weights). **This is the learning part.** In other words, 
$$L(x) = \sum_{(v, y) \in \boldsymbol{E}} \text{loss}(F(x, v) - y).$$
Calculus tells us that the minimizer of $L$ satisfies the following system of equations (there may be many solutions that satisfy this, hence we do not necessarily obtain the minimizer -- we just hope it's something "good enough"):

> **The partial derivatives of L with respect to the weights $x$ should be zero**: $$\boxed{\frac{\partial L}{\partial x} = 0 }$$

We solve the equation above, iteratively, using a modification of the gradient descent method called **stochastic gradient descent**. 

*Backpropagation* is a method to compute derivatives quickly, using the chain rule: 

$$\frac{dF}{dx} = \frac{d}{dx}(F_3(F_2(F_1 (x))) = \frac{dF_3}{dF_2}\vert_{F_2=F_2(F_1(x))} \frac{dF_2(F_1(x))}{dF_1}\vert_{F_1 = F_1(x)} \frac{dF_1(x)}{dx}\vert_x.$$

A convenient way to visualise how the function $F$ is computed from the weights $x_i$ is to use a **computational graph**. It separates the big computation into small steps, and we can find the derivative of each step (each computation) on the graph.

**Backpropagation** is a technique for optimizing parameters in a neural network. There are two types of backpropagation, depending on the relation between the number of inputs and outputs in the neural network: 
- *Forward-mode*: $F$ has few inputs, but have many outputs 
- *Backward-mode*: $F$ has many inputs, but have few outputs 

*Forward-mode* differentiation tracks how one input affects every node. *Reverse-mode* differentiation tracks how every node affects one output. That is, forward-mode differentiation applies the operator $\dfrac{\partial (.)}{\partial x}$ to every node, while reverse mode differentiation applies the operator $\dfrac{\partial F}{\partial (.)}$ to every node. The general rule is to sum over all possible paths from one node to the other, multiplying the derivatives on each edge of the path together. 

### Computational Graphs

<img src="handwritten.png" alt="Drawing" style="width: 300px;"/>

1. For the computational graph above, compute the the $x$ derivative of $F$ using both forward- and backward-mode. Assume that the initial values of $x, y$ are 2 and 3 resp.

2. At first it seems unbelievable that reorganising the computation can make such an enormous difference. Let's do an experiment with matrices: consider the product of 3 matrices: $A, B, C$. Which order? $AB$ first or $BC$ first? In other words, should we compute $(AB)C$ or $A(BC)$? 

Count the number of multiplications in each case and discuss what's the fastest strategy. This can be generalised to a product of $n$ matrices $A_1, A_2, \dots, A_n$. Both backpropagation, and the *chain matrix multiplication* problem are instances of dynamic programming problems. Colah's blog listed below also points out the connection between dynamic programming and back-propagation. 

***
Source: 
[1] http://colah.github.io/posts/2015-08-Backprop/
***

In [6]:
# let's implement the example from the previous open discussion in pytorch 
x = Variable(torch.tensor(2.), requires_grad=True)
y = Variable(torch.tensor(3.), requires_grad=True)
c = x**2 
s = x + y 
F = c*s 

In [7]:
F.backward(retain_graph=True)

In [8]:
F

tensor(20., grad_fn=<MulBackward0>)

In [9]:
F.backward()

This is the derivative of $F$ with respect to $x$! Remember that in the backward node, the derivative $\dfrac{\partial F}{\partial (.)}$ is moving backwards. So,
```python
x.grad
``` 
is the $x$-derivative of the output.

In [10]:
x.grad.numpy()

array(48., dtype=float32)

**Exercise**: 

Let $F = \log(x) + x^2 y + y^2$. 

Evaluate $\dfrac{\partial{F}}{\partial{x}}$ and $\dfrac{\partial{F}}{\partial{y}}$ at the point $x = 2$, $y = 3$ by hand (both in forward and backward modes) and by using `torch`.

In [11]:
# Define the variables x and y
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# Define the function F
F = torch.log(x) + x**2 * y + y**2

# Compute gradients
F.backward()

# Print the gradients
x_grad = x.grad
y_grad = y.grad

print('dF/dx:', x_grad.item())
print('dF/dy:', y_grad.item())

dF/dx: 12.5
dF/dy: 10.0


### The Initial Weights $x_0$ in Gradient Descent 

The architecture in a neural net decides the form of the learning function $F(x, v)$. The training data goes into $v$. Then we *initialize* the weights $x$ in the matrices $A$ and vectors $b$. From those initial weights $x_0$, the optimisation algorithm (normally a form of gradient descent) computes weights $x_1$, $x_2$ etc aiming to minimizing the total loss iteratively. 

The million-pounds question is: *What weights $x_0$ should we start with?* Choosing $x_0 = 0$ would be a disaster (why?). Poor initialisation is an important cause of failure in deep learning.  

Hanin and Rolnick [1] show that the initial variance $\sigma^2$ controls the mean of the computed weights. The layer widths controls the variance of the weights. The key point is this: 

> Many-layered depth can reduce the loss on the training set. But if $\sigma^2$ is wrong or width is sacrificed, then gradient descent can lose control of the weights. They can explode to infinity or implode to zero. 

Source:
[1] B. Hanin and D. Rolnick, *How to start training: The effect of initialisation and architecture*, arXiv: 1803.01719, 19/06/2018.

### Finding the best weights x: Gradient Descent and Stochastic Gradient Descent 

#### Gradient Descent toward the mininum 

> How to minimise a function $f(x_1, x_2, \dots, x_n)$? 

Calculus teaches us that all the first derivatives $\frac{\partial f}{\partial x_i}$ are zero at the minimum (when $f$ is smooth). If we have $n=20$ unknowns (a small number in deep learning) then minimising one function $f$ produces 20 equations. *Gradient-descent* uses the derivatives $\partial f/\partial x_i$ to find a direction that reduces $f(x)$. 

> The steepest direction in which $f(x)$ decreases the fastest, is given by $-\nabla f$: 
$$\boxed{\text{Gradient descent: } x_{k+1} = x_k - s_k \nabla f(x_k)}\qquad (\ast)$$

The symbol $\nabla f$ represents the vector of $n$ partial derivatives of $f$: its *gradient*.  

$$\boxed{\text{Gradient : } \nabla f(x_1, x_2, \dots, x_n) = \left(\frac{\partial F}{\partial x_1}, \frac{\partial F}{\partial x_2}, \dots, \frac{\partial F}{\partial x_n} \right) }$$



So the equation $(\ast)$ above is a vector equation for each step $k = 1, 2, \dots$ and $s_k$ is the  *stepsize* or the *learning rate*. We hope to move toward the point $x^{\ast}$ where the graph of $f(x)$ hits the bottom. 

#### Some examples 
1. For a constant vector $\mathbf{a} = (a_1, a_2, \dots, a_n)$, $F(\mathbf{x}) = \mathbf{a}^\intercal \mathbf{x}$ has gradient 
$\nabla F = \mathbf{a}.$

2. For a symmetric matrix $S$, the gradient of $F(\mathbf{x}) = \mathbf{x}^{\intercal} \mathbf{S} \mathbf{x}$ is  $\nabla F = 2 \mathbf{S} \mathbf{x}.$

3. For a positive definite symmetric matrix $S$, the minimum of a quadratic $F(x)=\frac{1}{2}x^{\intercal} S x - a^{\intercal}x$ is the negative number $F_min = - \frac{1}{2} a^{\intercal} S a$  at $x^{\ast} = S^{-1}a$.

4. Let $F(X) = \det (X)$, the determinant of a square matrix $X$. What do you think the partial derivative $\frac{\partial F}{\partial x_{ij}}$ looks like? 

#### Optimisation with zig-zagging
The example $f(x_1, x_2) = \frac{1}{2}(x^2 + by^2)$ is extremely useful for $0 < b <= 1$. 
1. Calculate the gradient $\nabla f$. 
2. We know that the minimum of $f$ is at $(0,0)$. Suppose instead we try to reach the minimum using the equation $(\ast)$ above with *exact line search*. That means that at each step we shall choose $s_k$ for which $f$ decreases the most. Show that: 
$$x_k = b \bigg (\frac{b-1}{b+1}\ \bigg)^k, y_k=\bigg(\frac{1-b}{1+b}\bigg)^k, f(x_k, y_k) = \bigg(\frac{1-b}{1+b}\bigg)^{2k}f(x_0,y_0),$$
where $(x_0, y_0) = (b, 1)$. 

### Stochastic Gradient Descent and ADAM (optional)

Gradient descent is fundamental in training a deep neural network. It is based on a step of the form 
$$x_{k+1} = x_k - s_k \nabla L(x_k).$$ That step should lead us downhill toward the point $x^{\ast}$ where the loss function $L(x)$ is minimised for the test data $v$. But for large networks with many samples in the training set, this algorithm (as it stands) is not sucessful! 

It's important to recognize two different problems with classical steepest descent: 

1. Computing $\nabla L$ (the gradient of the loss) function at every descent step - the derivatives of the total loss $L$ with respect to all the weights $x$ in the network - is too expensive. 

That total loss adds the individual losses *$l(x, v_i)$ for every sample $v_i$ in the training set* -- potentially millions of separate losses are computed and added in every computation of $L$.

2. The number of weights is even larger. So $\nabla_x L = 0$ for many different choices $x^{\ast}$ of the weights. **Some of those choices can give poor results on unseen test data.** The learning function $F$ can fail to "generalise". But **stochastic gradient descent** (SGD) does find weights $x^{\ast}$ that generalise. 

**Stochastic gradient descent uses only a "minibatch" of the training data at each step**. $B$ samples will be chosen randomly. Replacing the full batch of all the traiing data by a minibatch changes $L(x) = \frac{1}{n} \sum l_i(x)$ to a sum of only $B$ losses. This resolves both difficulties at once. The success of deep learning rests on these two facts: 

1. Computing $\nabla l_i$ by backpropagation on B samples is much faster. Often $B$ = 1. 
2. The stochastic algorithm produces weights $x^{\ast}$ that also succeed on unseen data. 

Something remarkable observed in practice is that the *SGD* avoids overfitting. Another fundamental strategy in training a neural network is **early stopping**. We'll provide more details in our practical tutorials. 

#### Stochastic Descent Using One Sample Per Step 

To simplify, suppose each minibatch contains only one sample $v_k$ (so B = 1). That sample is chosen randomly. The theory of stochastic descent usually assumes that the sample is replaced after use - in principle the sample could be chosen again at step k + 1. In practice, we often omit replacement and work through samples in a random order. 

Each pass through the training data is **one epoch** of the descent algorithm. Ordinary gradient descent computes one epoch per step (batch mode). Stochastic gradient descent needs many steps (for minibatches). The online advice is to choose $B \leq 32$. 

Stochastic descent began with a seminal paper by Robbins and Monro [1] where *they developed a fast method to converge to the desired optimum in probability*: 
> $\lim_{k\to\infty} Prob(\vert\vert x_k - x^{\ast}\vert\vert > \epsilon) \to 0.$

A word of caution: Stochastic descent is more sensitive to the stepsizes $s_k$ than full gradient descent. A typical feature of stochastic gradient descent is "semi-convergence": fast convergence at the start. Improvements we can use for facilitating the convergence of the SGD algorithm at late state iterations are 
1. adding *momentum* (e.g. Nesterov momentum etc)
2. adaptive *learning rates* (e.g. ADAM etc)

#### Fast convergence at the start: Least Squares with n = 1 

In this case the $i$-ith loss is $l_i = \frac{1}{2}(a_i x - b_i)^2$ with $a_i > 0$. The gradient of $l_i$ is its derivative $a_i(a_i x - b_i)$. The total loss over all $N$ samples is 
$$L(x) = \frac{1}{2N}\sum (a_i x - b_i)^2,$$
which is least squares with $N$ equations and 1 unknown. We can then compute the gradient: 

$$\nabla L = \frac{1}{N} \sum a_i (a_i x - b_i) = 0.$$ The solution is $x^{\ast} = \frac{\sum a_i b_i}{\sum a_i^2}.$

*Important:* If $B/A$ is the largest ratio $b_i/a_i$, then the true solution $x^{\ast}$ is below $B/A$. Similarly $x^{\ast}$ is above the smallest ratio $\beta/\alpha$. Therefore if $x_k$ is outside the interval $I$ from $\beta/\alpha$ to $B/A$, then the $k$-th gradient descent step will move toward that interval $I$ containing $x^{\ast}$.

In [12]:
# Code for one-dimensional least squares
from torch.autograd import Variable 

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# B is batch size; D_in is input dimension;
# N is sample size; D_out is output dimension.
B, N, D_in, D_out = 4, 20, 2, 1

# We're generating some synthetic data here.
# The weights to be learned are (w1, w2) = (1.0, 0.2)
# https://stackoverflow.com/questions/17869840/numpy-vector-n-1-dimension-n-dimension-conversion
eps = 1.e-2
xrange = yrange = np.arange(0.0, 1.0, 0.1)
g = np.meshgrid(xrange, yrange, sparse=False, indexing='ij')
_x = np.vstack(tup=tuple(map(np.ravel, g))).T
_w = np.array((0.4, 0.2)).reshape(1, -1).T
_y = _x.dot(_w) + eps * np.random.rand(_x.shape[0], 1)

# select a small sample of the data 
np.random.seed(42)
idx = np.random.randint(0, 100, N)
x_np = _x[idx]
y_np = _y[idx]

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.

x = Variable(torch.Tensor(x_np)) 
y = Variable(torch.Tensor(y_np)) 

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w = torch.randn(D_in, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-2
epochs = 500
weights = np.empty((epochs//10, 2))
losses = np.empty(epochs//10)
for t in range(epochs):
    sample = np.random.randint(0, 20, B)
    x_B, y_B = x[sample], y[sample]
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x_B.mm(w)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y_B).pow(2).sum()

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w -= learning_rate * w.grad # this is the gradient of loss with respect to w
        
        # Manually zero the gradients after updating weights
        w.grad.zero_()
        
    if t % 10 == 0: 
        ind = int(t/10)
        losses[ind] = loss.item()
        weights[ind, :] = w.data.view(1, -1).numpy()[0]
        print(t, losses[ind], weights[ind, :])

# compare this with the initial weights we had set up in our data 
print('final weights:', w)

0 16.81271743774414 [-0.02406471 -2.2378912 ]
10 5.905333995819092 [ 0.4373928  -1.79179537]
20 1.8725301027297974 [ 0.70951295 -1.54771721]
30 2.503507137298584 [ 0.83704448 -1.35265326]
40 1.4296228885650635 [ 0.90832788 -1.19260418]
50 0.24486276507377625 [ 0.97839952 -1.04354632]
60 0.21676556766033173 [ 0.99849898 -0.94356686]
70 0.5226307511329651 [ 1.00709164 -0.84442461]
80 0.2908508777618408 [ 1.00617743 -0.77556396]
90 0.3864205479621887 [ 1.00141335 -0.70546967]
100 0.5313148498535156 [ 0.96467346 -0.64023703]
110 0.19246816635131836 [ 0.94105804 -0.59466314]
120 0.17141762375831604 [ 0.93415409 -0.52809298]
130 0.14901241660118103 [ 0.90870672 -0.48698023]
140 0.030180707573890686 [ 0.90428048 -0.45444131]
150 0.09222254902124405 [ 0.8701337  -0.43882686]
160 0.15866123139858246 [ 0.83451885 -0.41398078]
170 0.022074252367019653 [ 0.81093192 -0.37333301]
180 0.09172672033309937 [ 0.79354519 -0.34541798]
190 0.18085487186908722 [ 0.7822423  -0.31247029]
200 0.224258109927177